In [16]:
!pip install pycuda

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 66 kB 4.6 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626633 sha256=1b91340b930a839db423ecc8928dd5160698d4d04ecfafdd133f364453b2632c
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.9-py2.py3-none-any.whl size=62370 sha256=30213a6399eb1317ad4f09d00c9e752cc170ae3f1385c84796aec1feac2f37df
  Stored in directory: /root/.cache/pip/wheels/41/b9/6e/94bb014f6484b15ec77e7877f3a227609481ffd98db364504d
Successfully built pycuda pytools


# Imports

In [1]:
import shutil
import random
import os

import cv2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from tabulate import tabulate
from sklearn.model_selection import train_test_split

In [2]:
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F

import pycuda.driver as cuda

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision.utils import make_grid

# Settings

In [3]:
cuda.init()
## Get Id of default device
torch.cuda.current_device()
# 0
cuda.Device(0).name() # '0' is the id of your GPU

'Tesla K80'

In [4]:
if not torch.cuda.is_available():
  raise Exception("GPU not availalbe. CPU training will be too slow.")
print("device name", torch.cuda.get_device_name(0))

device name Tesla K80


### Load data from personal drive
- data folder was created during the preprocessing within a local notebook 

In [5]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
artist_folder = '/content/drive/MyDrive/final'

# Preprocess data

In [7]:
dataset = ImageFolder(artist_folder, transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))

In [8]:
n = len(dataset) # n = 8356
len_train = 5000
len_dev = 1678 # (8356 - 5000) / 2
len_test = 1678

### Train, dev, test

In [9]:
train_data, dev_data, test_data = random_split(dataset,[len_train, len_dev, len_test])

### Create batches

In [10]:
batch_size = 256

train = DataLoader(train_data, batch_size = batch_size, shuffle = True, num_workers = 2)
dev = DataLoader(dev_data, batch_size = batch_size * 2, num_workers = 2)
test = DataLoader(test_data, batch_size = batch_size * 2, num_workers = 2)

# Data modelling

In [11]:
class ArtistClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        images = images.cuda()
        labels = labels.cuda()
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        images = images.cuda()
        labels = labels.cuda()
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [19]:
class ConvolutionalNN(ArtistClassificationBase):
    def __init__(self, n):
        super().__init__()

        kernel_size = 3
        pool_size = 3

        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = kernel_size, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size = kernel_size, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(pool_size,pool_size),

            nn.Conv2d(64, 128, kernel_size = kernel_size, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size = kernel_size, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(pool_size,pool_size),

            nn.Conv2d(128, 256, kernel_size = kernel_size, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size = kernel_size, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(pool_size,pool_size),
            
            nn.Flatten(),
            nn.Linear(6400,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,n)
        )
    
    def forward(self, xb):
        return self.network(xb)

from torchsummary import summary
model = ConvolutionalNN(n).cuda()
summary(model, input_size=(3, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 150, 150]             896
              ReLU-2         [-1, 32, 150, 150]               0
            Conv2d-3         [-1, 64, 150, 150]          18,496
              ReLU-4         [-1, 64, 150, 150]               0
         MaxPool2d-5           [-1, 64, 50, 50]               0
            Conv2d-6          [-1, 128, 50, 50]          73,856
              ReLU-7          [-1, 128, 50, 50]               0
            Conv2d-8          [-1, 128, 50, 50]         147,584
              ReLU-9          [-1, 128, 50, 50]               0
        MaxPool2d-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 256, 16, 16]         295,168
             ReLU-12          [-1, 256, 16, 16]               0
           Conv2d-13          [-1, 256, 16, 16]         590,080
             ReLU-14          [-1, 256,

In [13]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)
  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for i, batch in enumerate(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            print(f'Batch Nr: {i + 1} done', end = '\r')
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

In [14]:
num_epochs = 50
opt_func = torch.optim.Adam
lr = 0.01
n = len(train.dataset.dataset.classes)

model = ConvolutionalNN(n).cuda()

#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train, dev, opt_func)

Epoch [0], train_loss: 148.7300, val_loss: 3.6776, val_acc: 0.0985
Epoch [1], train_loss: 3.6143, val_loss: 3.6225, val_acc: 0.0997
Epoch [2], train_loss: 3.5401, val_loss: 3.4746, val_acc: 0.1210
Epoch [3], train_loss: 3.4120, val_loss: 3.4089, val_acc: 0.1242
Epoch [4], train_loss: 3.3377, val_loss: 3.3974, val_acc: 0.1256
Epoch [5], train_loss: 3.3096, val_loss: 3.3869, val_acc: 0.1408
Epoch [6], train_loss: 3.3179, val_loss: 3.5969, val_acc: 0.1345
Epoch [7], train_loss: 3.3481, val_loss: 3.3670, val_acc: 0.1267
Epoch [8], train_loss: 3.3201, val_loss: 3.3858, val_acc: 0.1339
Epoch [9], train_loss: 3.2439, val_loss: 3.3389, val_acc: 0.1525
Epoch [10], train_loss: 3.2368, val_loss: 3.3524, val_acc: 0.1349
Epoch [11], train_loss: 3.2368, val_loss: 3.3626, val_acc: 0.1423
Epoch [12], train_loss: 3.2257, val_loss: 3.3488, val_acc: 0.1440
Epoch [13], train_loss: 3.1750, val_loss: 3.3140, val_acc: 0.1371
Epoch [14], train_loss: 3.1571, val_loss: 3.3038, val_acc: 0.1551
Epoch [15], train_

KeyboardInterrupt: ignored